<a href="https://colab.research.google.com/github/Farii01/Internship_PINN_work/blob/main/PINN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
!pip install git+https://gricad-gitlab.univ-grenoble-alpes.fr/claveaur/pygeotools.git


  Cloning https://gricad-gitlab.univ-grenoble-alpes.fr/claveaur/pygeotools.git to /tmp/pip-req-build-x2cgbu__
  Running command git clone --filter=blob:none --quiet https://gricad-gitlab.univ-grenoble-alpes.fr/claveaur/pygeotools.git /tmp/pip-req-build-x2cgbu__
  Resolved https://gricad-gitlab.univ-grenoble-alpes.fr/claveaur/pygeotools.git to commit 87d5eab82dbae0c55a8d93113cd4ac6db38a0bf0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [121]:
import pygeotools
import torch
import torch.nn as nn
import numpy as np

In [122]:
model_path = "/content/COVOBS-x2_400reals.hdf5"


In [123]:
import pygeotools

pygeo = pygeotools.pygeotools()
model_name=  "COVOBS-x2_400reals.hdf5"
pygeo.loadModel(
    modelName="COVOBS-x2_400reals.hdf5",
    modelType="covobs_hdf5",
    modelPath=model_path
)

pygeo.isLoaded("COVOBS-x2_400reals.hdf5")  # Should return True


pygeotools was initialized with `verbose=True`.


True


*   **2 Epochs**
*   **2015 data**




# **Familiarizing with pygeo**

In [124]:
dir(pygeo)


['Error',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__ior__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__ror__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_pygeotools__colors',
 '_pygeotools__computedOperators',
 '_pygeotools__figures',
 '_pygeotools__gridTypes',
 '_pygeotools__grids',
 '_pygeotools__modelTypes',
 '_pygeotools__numericalCostLimit',
 '_pygeotools__numericalPrecision',
 '_pygeotools__observationTypes',
 '_pygeotools__observatories',
 '_pygeotools__precisions',
 '_pygeotools__verbose',
 'addFigure',
 'addGrid',
 'addMeasure',
 'addOperator',
 'addSubplot',
 'applyPrecisio

In [125]:
help(pygeo.addMeasure)

Help on method addMeasure in module pygeotools.pygeotools:

addMeasure(modelName: str, measureName: str, context={}) -> numpy.ndarray method of pygeotools.pygeotools.pygeotools instance
    # Adding a measure with a specific context



In [126]:
help(pygeo.getQuantities) # What the function expects, What it returns, What we can do with it


Help on method getQuantities in module pygeotools.pygeotools:

getQuantities(modelName: str) -> set method of pygeotools.pygeotools.pygeotools instance
    # Retrieving the available quantities of a model



In [127]:
pygeo[model_name].keys() # what is inside models


dict_keys(['type', 'path', 'quantities', 'measures'])

In [128]:
pygeo[model_name]['quantities'].keys()# inside quantities in models

dict_keys(['dgnm', 'dq10', 'gnm', 'q10', 'times'])

In [129]:
pygeo.getQuantities(model_name)


dict_keys(['dgnm', 'dq10', 'gnm', 'q10', 'times'])

In [130]:
# i was seeing the range
info  = pygeo.getQuantity(model_name, "times")
print(f"Model covers from {info .min()} to {info.max()}")


Model covers from 1840.0 to 2022.0


In [131]:
times = pygeo.getQuantity(model_name, "times")
print(times[0:1])

[1840.]


Focusing from

In [132]:
# Define context: surface of Earth's core, max degree 14
context = {
    "r": pygeo.constants["rEarth"],
    "lmax": 14
}

# Compute measures: Br (MF) and dBr/dt (SV). u.
pygeo.addMeasure(modelName=model_name, measureName="MF", context= context) # theta phy br
pygeo.addMeasure(modelName=model_name, measureName="SV", context=context) # the phi and the change in br

MF = pygeo.getMeasure(model_name, "MF", context=context)
SV = pygeo.getMeasure(model_name, "SV", context=context)

14


In [133]:
import torch
import torch.nn as nn

class CoreFlowPINN(nn.Module):
    def __init__(self):
        super(CoreFlowPINN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 32),
            nn.Tanh(),
            nn.Linear(32, 32),
            nn.Tanh(),
            nn.Linear(32, 2)  # Output: u_theta and u_phi
        )

    def forward(self, x):
        return self.net(x)


In [144]:
find_years = range(2020, 2026)

# using pygeo to fetch time data
years = pygeo.getQuantity(model_name, "times")

for year in find_years:
    if year in years:
        mf_grid = pygeo.selectFromMeasure(
            modelName=model_name,
            measure=MF,
            options={"time": (year, year)}
        )

        sv_grid = pygeo.selectFromMeasure(
            modelName=model_name,
            measure=SV,
            options={"time": (year, year)}
        )

        print(f"\n Data loaded for year {year} \n")

        # Flatten grid to [theta, phi, value]
        mf_np = np.array(mf_grid).reshape(-1, 3)
        sv_np = np.array(sv_grid).reshape(-1, 3)

        # Convert to torch tensors
        theta = torch.tensor(mf_np[:, 0:1], dtype=torch.float32, requires_grad=True)
        phi   = torch.tensor(mf_np[:, 1:2], dtype=torch.float32, requires_grad=True)
        Br    = torch.tensor(mf_np[:, 2:3], dtype=torch.float32)
        dBr_dt = torch.tensor(sv_np[:, 2:3], dtype=torch.float32)

        inputs = torch.cat([theta, phi], dim=1)

        # Reinitialize model and optimizer for each year
        model = CoreFlowPINN()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

        # Training loop for current year
        for epoch in range(11):  # or just range(1) for 1 epoch
            optimizer.zero_grad()

            u_pred = model(inputs)
            u_theta = u_pred[:, 0:1]
            u_phi = u_pred[:, 1:2]

            q_theta = u_theta * Br
            q_phi = u_phi * Br

            dq_theta_dtheta = torch.autograd.grad(
                q_theta, theta,
                grad_outputs=torch.ones_like(q_theta),
                create_graph=True, retain_graph=True
            )[0]

            dq_phi_dphi = torch.autograd.grad(
                q_phi, phi,
                grad_outputs=torch.ones_like(q_phi),
                create_graph=True, retain_graph=True
            )[0]

            residual = dBr_dt + dq_theta_dtheta + dq_phi_dphi
            loss = torch.mean(residual**2)

            loss.backward()
            optimizer.step()

            print(f"{year} : Epoch {epoch} --- Loss {loss.item():.6f}")

    else:
        print(f"\nNo data available for year {year}")



 Data loaded for year 2020 

2020 : Epoch 0 --- Loss 1755.828735
2020 : Epoch 1 --- Loss 1749.650879
2020 : Epoch 2 --- Loss 1756.939087
2020 : Epoch 3 --- Loss 1749.621948
2020 : Epoch 4 --- Loss 1753.254272
2020 : Epoch 5 --- Loss 1750.293213
2020 : Epoch 6 --- Loss 1746.247070
2020 : Epoch 7 --- Loss 1749.823730
2020 : Epoch 8 --- Loss 1749.614380
2020 : Epoch 9 --- Loss 1748.474121
2020 : Epoch 10 --- Loss 1749.426514

 Data loaded for year 2021 

2021 : Epoch 0 --- Loss 1662.868408
2021 : Epoch 1 --- Loss 1655.493042
2021 : Epoch 2 --- Loss 1662.379272
2021 : Epoch 3 --- Loss 1657.233032
2021 : Epoch 4 --- Loss 1711.221069
2021 : Epoch 5 --- Loss 1655.474121
2021 : Epoch 6 --- Loss 1691.586670
2021 : Epoch 7 --- Loss 1656.705811
2021 : Epoch 8 --- Loss 1656.059082
2021 : Epoch 9 --- Loss 1655.478760
2021 : Epoch 10 --- Loss 1655.581787

 Data loaded for year 2022 

2022 : Epoch 0 --- Loss 1425.097778
2022 : Epoch 1 --- Loss 1400.092163
2022 : Epoch 2 --- Loss 1436.380859
2022 : E